<a href="https://colab.research.google.com/github/aninsung/Machine-Learning-Programming/blob/main/4%EC%A3%BC%EC%B0%A8_mhealth_cnn%2Blstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (오늘 실습과제)

Mhealth dataset으로 인간행동 분류하기

일단 데이터 읽어서 subject 번호가 컬럼면에 포함된 통합 data frame 만들고 CNN, LSTM, CNN-LSTM 해보기.

1. LLM(sLLM)을 제외한 어떤 모델을 써도 가능. 단 라이브러리 import 불가. 모델의 풀 소스가 있어야 함.

 2. 10명을 1+2, 3+4, 4+5,.... 5개그룹으로 나눠 2명씩 테스트 데이터로 하면서 cross validation.

 3. 데이터 증대는 안됨.

4. 오직 모델로만 성능 향상

5. 상위 30%씩 차등.

6. 성적에 반영

7. 11월15일까지. Github게시

교차 평가로 평균 F1 기준.

8. 본인이 완벽하게 소스 및 모델을 설명할 수 있어야 함. GPT를 쓰던 뭘 쓰던 그건 상관안함. 재현성이 보장되어야 함.
--------------------------------------------------------------------------
--------------------------------------------------------------------------

## 데이터설명
📌 Dataset Summary

Subjects: 10명 (다양한 프로필의 자원자)

Activities: 12개

Sensors: 3개 (가슴, 오른쪽 손목, 왼쪽 발목 → elastic strap으로 고정)

측정 정보:

Acceleration (가속도)

Gyroscope (회전율)

Magnetometer (자기장 방향)

Chest sensor: 2-lead ECG 추가 (활동 인식에는 사용 안 함, 향후 연구용)

Sampling Rate: 50 Hz

환경: Out-of-lab, 제약 없이 자유롭게 활동 수행 (단, 최선을 다해 실행)

📌 Activity Set (12 classes)

정적 활동: Standing still, Sitting, Lying down

일상/이동 활동: Walking, Climbing stairs, Cycling

운동/역동적 활동: Jogging, Running, Jump front & back

반복 동작: Waist bends, Arm elevation, Knees bending

📌 특징

ECG 포함: 운동 중 심전도 기록 → 부정맥, 심박수 모니터링, 운동 효과 분석 가능

다양성 확보:

동작 강도 차이 (예: Sitting vs. Running)

속도 차이 (예: Walking vs. Jogging)

다른 신체 부위 동작 (예: Arm elevation vs. Knees bending)

실생활 반영: 실제 환경에서 수행되어 일반화 성능 높음

In [1]:
import pandas as pd
import glob
import numpy as np

# 1. 파일 리스트
files = sorted(glob.glob("/content/drive/MyDrive/Colab Notebooks/mhealth+dataset/MHEALTHDATASET/mHealth_subject*.log"))

# 2. 컬럼 정의
columns = [
    "chest_acc_x","chest_acc_y","chest_acc_z",
    "ecg_1","ecg_2",
    "chest_gyro_x","chest_gyro_y","chest_gyro_z",
    "chest_mag_x","chest_mag_y","chest_mag_z",
    "ankle_acc_x","ankle_acc_y","ankle_acc_z",
    "ankle_gyro_x","ankle_gyro_y","ankle_gyro_z",
    "ankle_mag_x","ankle_mag_y","ankle_mag_z",
    "wrist_acc_x","wrist_acc_y","wrist_acc_z",
    "label"
]

# 3. 각 파일 읽어서 subject_id 추가
dfs = []
for i, file in enumerate(files, start=1):
    df = pd.read_csv(file, sep="\s+", header=None)
    df.columns = columns
    df["subject_id"] = i
    dfs.append(df)

# 4. 통합 DataFrame
data = pd.concat(dfs, ignore_index=True)


<>:24: SyntaxWarning: invalid escape sequence '\s'
<>:24: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-4155912323.py:24: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file, sep="\s+", header=None)


KeyboardInterrupt: 

In [ ]:
def create_windows(df, frame_size=100, hop_size=50):
    X, y = [], []
    data_values = df.drop(["label", "subject_id"], axis=1).values
    labels = df["label"].values
    for i in range(0, len(data_values)-frame_size, hop_size):
        X.append(data_values[i:i+frame_size])
        y.append(labels[i+frame_size-1])  # 마지막 시점 label
    return np.array(X), np.array(y)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dense, Dropout

def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    # 첫 번째 Conv
    x = Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    # 두 번째 Conv
    x = Conv1D(filters, kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    # shortcut 차원 맞추기
    if shortcut.shape[-1] != filters or stride != 1:
        shortcut = Conv1D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def build_resnet(timesteps, n_features, n_classes):
    inputs = Input(shape=(timesteps, n_features))

    x = Conv1D(64, 7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Residual blocks
    x = residual_block(x, 64)
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 256, stride=2)

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(n_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
from sklearn.metrics import f1_score
import numpy as np

# 결과 저장용 리스트
acc_list = []
loss_list = []
f1_list = []

subjects = np.arange(1,11)
folds = [(subjects[i], subjects[i+1]) for i in range(0,10,2)]

# Define parameters for windowing and model
timesteps = 100
n_features = data.drop(["label", "subject_id"], axis=1).shape[1]
n_classes = data["label"].nunique()

for fold_idx, test_subjects in enumerate(folds, start=1):
    # Train/Test 분리
    test_data = data[data["subject_id"].isin(test_subjects)]
    train_data = data[~data["subject_id"].isin(test_subjects)]

    # 윈도우 생성
    X_train, y_train = create_windows(train_data, frame_size=timesteps, hop_size=50)
    X_test, y_test = create_windows(test_data, frame_size=timesteps, hop_size=50)

    print(f"\n=== Fold {fold_idx}: Test subjects {test_subjects} ===")
    print("Samples:", X_train.shape[0], "(train)", X_test.shape[0], "(test)")

    # 모델
    model = build_resnet(timesteps, n_features, n_classes)
    # 학습
    model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)

    # 평가
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # F1-score 계산 (다중 클래스는 'macro' 추천)
    f1 = f1_score(y_test, y_pred_classes, average='macro')

    print(f"Test Accuracy: {acc:.4f}, Loss: {loss:.4f}, F1-score: {f1:.4f}")

    # 리스트에 저장
    acc_list.append(acc)
    loss_list.append(loss)
    f1_list.append(f1)

# Cross-validation 평균
print("\n=== Cross-validation Average ===")
print(f"Average Accuracy: {np.mean(acc_list):.4f}")
print(f"Average Loss: {np.mean(loss_list):.4f}")
print(f"Average F1-score: {np.mean(f1_list):.4f}")